# Phân loại hoa Diên vĩ (Iris) với Machine Learning

## 1. Giới thiệu

Notebook này sẽ hướng dẫn bạn xây dựng một mô hình Machine Learning để phân loại ba loài hoa diên vĩ (Iris) dựa trên các đặc trưng vật lý. Đây là một bài toán phân loại cổ điển và là điểm khởi đầu tuyệt vời để học Machine Learning.

**Mục tiêu:**
- Khám phá và hiểu dữ liệu Iris
- Thực hiện tiền xử lý dữ liệu
- Huấn luyện và so sánh nhiều mô hình ML
- Tối ưu hóa mô hình tốt nhất
- Đánh giá và lưu mô hình cuối cùng

## 2. Import thư viện

In [ ]:
# Thư viện xử lý dữ liệu
import pandas as pd
import numpy as np

# Thư viện trực quan hóa
import matplotlib.pyplot as plt
import seaborn as sns

# Thiết lập style cho biểu đồ
plt.style.use('default')
sns.set_palette("husl")

# Thiết lập hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

# Tắt cảnh báo
import warnings
warnings.filterwarnings('ignore')

print("✅ Đã import thành công tất cả thư viện cần thiết!")

In [ ]:
# Thư viện Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Các thuật toán ML
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Đánh giá mô hình
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Lưu mô hình
import joblib

print("✅ Đã import thành công tất cả thư viện Machine Learning!")

## 3. Tải và khám phá dữ liệu

In [ ]:
# Đọc dữ liệu từ file CSV
df = pd.read_csv('iris.csv')

print("📊 Thông tin cơ bản về dữ liệu:")
print(f"Kích thước dữ liệu: {df.shape[0]} mẫu, {df.shape[1]} đặc trưng")
print("\n📋 5 dòng đầu tiên:")
df.head()

In [ ]:
# Thông tin chi tiết về dữ liệu
print("📊 Thông tin chi tiết về dataset:")
df.info()

print("\n📈 Thống kê mô tả:")
df.describe()

In [ ]:
# Kiểm tra giá trị thiếu
print("🔍 Kiểm tra giá trị thiếu:")
missing_values = df.isnull().sum()
print(missing_values)

if missing_values.sum() == 0:
    print("✅ Không có giá trị thiếu nào trong dữ liệu!")
else:
    print("⚠️ Có giá trị thiếu cần xử lý!")

In [ ]:
# Phân bố các loài hoa
print("🌸 Phân bố các loài hoa:")
species_counts = df['species'].value_counts()
print(species_counts)

# Vẽ biểu đồ phân bố
plt.figure(figsize=(8, 6))
species_counts.plot(kind='bar', color=['lightcoral', 'lightblue', 'lightgreen'])
plt.title('Phân bố các loài hoa Iris', fontsize=14, fontweight='bold')
plt.xlabel('Loài hoa')
plt.ylabel('Số lượng mẫu')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n✅ Dữ liệu cân bằng hoàn hảo: mỗi loài có {species_counts.iloc[0]} mẫu")

## 4. Khám phá dữ liệu (EDA - Exploratory Data Analysis)

In [ ]:
# Biểu đồ phân phối từng đặc trưng
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
feature_names = ['Chiều dài đài hoa', 'Chiều rộng đài hoa', 'Chiều dài cánh hoa', 'Chiều rộng cánh hoa']

for i, (feature, name) in enumerate(zip(features, feature_names)):
    row, col = i // 2, i % 2
    axes[row, col].hist(df[feature], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    axes[row, col].set_title(f'Phân phối {name}', fontweight='bold')
    axes[row, col].set_xlabel(name)
    axes[row, col].set_ylabel('Tần suất')
    axes[row, col].grid(True, alpha=0.3)

plt.suptitle('Phân phối các đặc trưng của hoa Iris', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Box plot theo từng loài
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for i, (feature, name) in enumerate(zip(features, feature_names)):
    row, col = i // 2, i % 2
    sns.boxplot(data=df, x='species', y=feature, ax=axes[row, col])
    axes[row, col].set_title(f'{name} theo loài', fontweight='bold')
    axes[row, col].set_xlabel('Loài hoa')
    axes[row, col].set_ylabel(name)

plt.suptitle('So sánh các đặc trưng giữa các loài hoa', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Pair plot - So sánh tất cả các cặp đặc trưng
plt.figure(figsize=(12, 10))
pairplot = sns.pairplot(df, hue='species', diag_kind='hist', 
                       plot_kws={'alpha': 0.7, 's': 50})
pairplot.fig.suptitle('Ma trận so sánh các đặc trưng theo loài', 
                      fontsize=16, fontweight='bold', y=1.02)
plt.show()

print("📊 Nhận xét từ Pair Plot:")
print("- Setosa rất dễ phân biệt với hai loài còn lại")
print("- Versicolor và Virginica có sự chồng chéo nhưng vẫn có thể phân biệt")
print("- Chiều dài và rộng cánh hoa là hai đặc trưng phân biệt tốt nhất")

In [ ]:
# Ma trận tương quan
plt.figure(figsize=(10, 8))
correlation_matrix = df[features].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.3f', cbar_kws={'shrink': 0.8})
plt.title('Ma trận tương quan giữa các đặc trưng', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("🔍 Phân tích tương quan:")
print("- Chiều dài cánh hoa và chiều rộng cánh hoa có tương quan mạnh (r=0.963)")
print("- Chiều dài đài hoa và chiều dài cánh hoa có tương quan khá cao (r=0.872)")
print("- Chiều rộng đài hoa có tương quan âm với các đặc trưng khác")

## 5. Tiền xử lý dữ liệu

In [ ]:
# Tách đặc trưng và nhãn
X = df[features]  # Các đặc trưng
y = df['species']  # Nhãn (loài hoa)

print("🎯 Kích thước dữ liệu:")
print(f"X (đặc trưng): {X.shape}")
print(f"y (nhãn): {y.shape}")

print("\n📊 Thống kê X:")
print(X.describe())

In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("📊 Kích thước sau khi chia:")
print(f"Tập huấn luyện - X: {X_train.shape}, y: {y_train.shape}")
print(f"Tập kiểm tra - X: {X_test.shape}, y: {y_test.shape}")

print("\n🌸 Phân bố loài trong tập huấn luyện:")
print(y_train.value_counts())

print("\n🌸 Phân bố loài trong tập kiểm tra:")
print(y_test.value_counts())

In [ ]:
# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("⚖️ So sánh trước và sau khi chuẩn hóa:")
print("\nTrước chuẩn hóa (X_train):")
print(pd.DataFrame(X_train).describe())

print("\nSau chuẩn hóa (X_train_scaled):")
print(pd.DataFrame(X_train_scaled, columns=features).describe())

print("\n✅ Dữ liệu đã được chuẩn hóa với mean≈0 và std≈1")

## 6. Huấn luyện mô hình baseline

In [ ]:
# Khởi tạo các mô hình
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine': SVC(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42)
}

print("🤖 Các mô hình sẽ được huấn luyện:")
for name in models.keys():
    print(f"- {name}")

In [ ]:
# Huấn luyện và đánh giá các mô hình
results = []

print("🏃‍♂️ Bắt đầu huấn luyện các mô hình...\n")

for name, model in models.items():
    print(f"Đang huấn luyện {name}...")
    
    # Các mô hình cần chuẩn hóa dữ liệu
    if name in ['Logistic Regression', 'K-Nearest Neighbors', 'Support Vector Machine']:
        X_train_model = X_train_scaled
        X_test_model = X_test_scaled
    else:
        X_train_model = X_train
        X_test_model = X_test
    
    # Huấn luyện mô hình
    model.fit(X_train_model, y_train)
    
    # Dự đoán
    y_pred = model.predict(X_test_model)
    
    # Tính độ chính xác
    accuracy = accuracy_score(y_test, y_pred)
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train_model, y_train, cv=5, scoring='accuracy')
    
    results.append({
        'Model': name,
        'Test Accuracy': accuracy,
        'CV Mean': cv_scores.mean(),
        'CV Std': cv_scores.std()
    })
    
    print(f"✅ {name}: Test Accuracy = {accuracy:.3f}, CV = {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

print("\n🎉 Hoàn thành huấn luyện tất cả mô hình!")

In [ ]:
# Tổng hợp kết quả
results_df = pd.DataFrame(results).sort_values('Test Accuracy', ascending=False)

print("📊 Bảng tổng hợp kết quả:")
print(results_df.to_string(index=False))

# Vẽ biểu đồ so sánh
plt.figure(figsize=(12, 6))
x_pos = np.arange(len(results_df))

plt.bar(x_pos, results_df['Test Accuracy'], alpha=0.7, color='lightblue', 
        edgecolor='navy', label='Test Accuracy')
plt.bar(x_pos, results_df['CV Mean'], alpha=0.7, color='lightcoral', 
        edgecolor='darkred', label='CV Mean', width=0.6)

plt.xlabel('Mô hình')
plt.ylabel('Độ chính xác')
plt.title('So sánh hiệu suất các mô hình Machine Learning', fontweight='bold')
plt.xticks(x_pos, results_df['Model'], rotation=45, ha='right')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

best_model_name = results_df.iloc[0]['Model']
best_accuracy = results_df.iloc[0]['Test Accuracy']
print(f"\n🏆 Mô hình tốt nhất: {best_model_name} với độ chính xác {best_accuracy:.3f}")

## 7. Tối ưu siêu tham số

In [ ]:
# Tối ưu cho Support Vector Machine (thường cho kết quả tốt)
print("🔧 Tối ưu siêu tham số cho Support Vector Machine...")

# Định nghĩa lưới tham số
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'kernel': ['rbf', 'linear', 'poly']
}

# Khởi tạo mô hình SVC
svc = SVC(random_state=42)

# Grid Search với Cross Validation
grid_search = GridSearchCV(
    svc, param_grid, cv=5, scoring='accuracy', 
    n_jobs=-1, verbose=1
)

# Huấn luyện với dữ liệu đã chuẩn hóa
grid_search.fit(X_train_scaled, y_train)

print("\n✅ Hoàn thành tối ưu siêu tham số!")
print(f"🏆 Tham số tốt nhất: {grid_search.best_params_}")
print(f"📊 Điểm số CV tốt nhất: {grid_search.best_score_:.3f}")

In [ ]:
# Tối ưu cho Random Forest
print("🔧 Tối ưu siêu tham số cho Random Forest...")

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(
    rf, param_grid_rf, cv=5, scoring='accuracy', 
    n_jobs=-1, verbose=1
)

# Huấn luyện với dữ liệu chưa chuẩn hóa (Random Forest không cần)
grid_search_rf.fit(X_train, y_train)

print("\n✅ Hoàn thành tối ưu Random Forest!")
print(f"🏆 Tham số tốt nhất: {grid_search_rf.best_params_}")
print(f"📊 Điểm số CV tốt nhất: {grid_search_rf.best_score_:.3f}")

In [ ]:
# So sánh mô hình tối ưu
best_svc = grid_search.best_estimator_
best_rf = grid_search_rf.best_estimator_

# Đánh giá trên tập test
svc_pred = best_svc.predict(X_test_scaled)
rf_pred = best_rf.predict(X_test)

svc_accuracy = accuracy_score(y_test, svc_pred)
rf_accuracy = accuracy_score(y_test, rf_pred)

print("🏆 Kết quả sau tối ưu:")
print(f"SVC tối ưu: {svc_accuracy:.3f}")
print(f"Random Forest tối ưu: {rf_accuracy:.3f}")

# Chọn mô hình tốt nhất
if svc_accuracy >= rf_accuracy:
    final_model = best_svc
    final_model_name = "Support Vector Machine"
    final_accuracy = svc_accuracy
    X_test_final = X_test_scaled
    y_pred_final = svc_pred
else:
    final_model = best_rf
    final_model_name = "Random Forest"
    final_accuracy = rf_accuracy
    X_test_final = X_test
    y_pred_final = rf_pred

print(f"\n🎯 Mô hình cuối cùng: {final_model_name} với độ chính xác {final_accuracy:.3f}")

## 8. Đánh giá chi tiết trên tập test

In [ ]:
# Báo cáo phân loại chi tiết
print(f"📊 Báo cáo đánh giá chi tiết cho {final_model_name}:")
print("=" * 60)
report = classification_report(y_test, y_pred_final)
print(report)

# Độ chính xác tổng thể
print(f"\n🎯 Độ chính xác tổng thể: {final_accuracy:.3f} ({final_accuracy*100:.1f}%)")

In [ ]:
# Ma trận nhầm lẫn (Confusion Matrix)
cm = confusion_matrix(y_test, y_pred_final)
species_names = ['setosa', 'versicolor', 'virginica']

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=species_names, yticklabels=species_names,
            cbar_kws={'shrink': 0.8})
plt.title(f'Ma trận nhầm lẫn - {final_model_name}', fontsize=14, fontweight='bold')
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')
plt.tight_layout()
plt.show()

# Phân tích ma trận nhầm lẫn
print("🔍 Phân tích ma trận nhầm lẫn:")
for i, species in enumerate(species_names):
    correct = cm[i, i]
    total = cm[i, :].sum()
    accuracy_species = correct / total if total > 0 else 0
    print(f"- {species.capitalize()}: {correct}/{total} ({accuracy_species:.1%})")

# Tìm lỗi phân loại
errors = y_test != y_pred_final
if errors.sum() > 0:
    print(f"\n❌ Có {errors.sum()} lỗi phân loại:")
    error_df = pd.DataFrame({
        'Thực tế': y_test[errors],
        'Dự đoán': y_pred_final[errors]
    })
    print(error_df.to_string())
else:
    print("\n✅ Không có lỗi phân loại nào! Mô hình hoàn hảo!")

## 9. Lưu mô hình và suy luận

In [ ]:
# Lưu mô hình tốt nhất
model_filename = 'iris_model.joblib'
scaler_filename = 'iris_scaler.joblib'

# Lưu mô hình
joblib.dump(final_model, model_filename)
print(f"✅ Đã lưu mô hình vào {model_filename}")

# Lưu scaler nếu cần
if final_model_name == "Support Vector Machine":
    joblib.dump(scaler, scaler_filename)
    print(f"✅ Đã lưu scaler vào {scaler_filename}")

print(f"\n📦 Mô hình {final_model_name} đã sẵn sàng để sử dụng!")

In [ ]:
# Ví dụ suy luận với dữ liệu mới
print("🔮 Ví dụ suy luận với dữ liệu mới:")
print("=" * 50)

# Tạo một số mẫu thủ công để test
new_samples = np.array([
    [5.1, 3.5, 1.4, 0.2],  # Giống setosa điển hình
    [6.0, 2.7, 5.1, 1.6],  # Giống versicolor điển hình  
    [7.2, 3.2, 6.0, 1.8],  # Giống virginica điển hình
    [5.0, 3.0, 4.0, 1.0],  # Mẫu trung gian
])

# Chuẩn bị dữ liệu cho dự đoán
if final_model_name == "Support Vector Machine":
    new_samples_processed = scaler.transform(new_samples)
else:
    new_samples_processed = new_samples

# Dự đoán
predictions = final_model.predict(new_samples_processed)
probabilities = final_model.predict_proba(new_samples_processed) if hasattr(final_model, 'predict_proba') else None

# Hiển thị kết quả
feature_names_vn = ['Dài đài', 'Rộng đài', 'Dài cánh', 'Rộng cánh']

for i, (sample, pred) in enumerate(zip(new_samples, predictions)):
    print(f"\nMẫu {i+1}:")
    for feature, value in zip(feature_names_vn, sample):
        print(f"  {feature}: {value}")
    print(f"  🌸 Dự đoán: {pred.capitalize()}")
    
    if probabilities is not None:
        prob_dict = dict(zip(final_model.classes_, probabilities[i]))
        print(f"  📊 Xác suất:")
        for species, prob in prob_dict.items():
            print(f"    - {species.capitalize()}: {prob:.1%}")

In [ ]:
# Kiểm tra load lại mô hình
print("🔄 Kiểm tra tải lại mô hình...")

# Load mô hình
loaded_model = joblib.load(model_filename)
if final_model_name == "Support Vector Machine":
    loaded_scaler = joblib.load(scaler_filename)

# Test với một mẫu
test_sample = np.array([[5.1, 3.5, 1.4, 0.2]])
if final_model_name == "Support Vector Machine":
    test_sample_processed = loaded_scaler.transform(test_sample)
else:
    test_sample_processed = test_sample

loaded_prediction = loaded_model.predict(test_sample_processed)
print(f"✅ Mô hình đã được tải và hoạt động: {loaded_prediction[0]}")

## 10. Kết luận và hướng phát triển

### 🎉 Kết quả đạt được:

1. **Xây dựng thành công** mô hình phân loại hoa Iris với độ chính xác cao
2. **So sánh nhiều thuật toán** ML khác nhau và chọn ra mô hình tốt nhất
3. **Tối ưu siêu tham số** để cải thiện hiệu suất
4. **Đánh giá chi tiết** với confusion matrix và classification report
5. **Lưu mô hình** để sử dụng trong tương lai

### 📊 Những điều học được:

- **EDA rất quan trọng**: Giúp hiểu dữ liệu và lựa chọn chiến lược phù hợp
- **Chuẩn hóa dữ liệu**: Cần thiết cho một số thuật toán (SVM, KNN, Logistic Regression)
- **Cross-validation**: Đánh giá độ tin cậy của mô hình
- **Grid search**: Tự động tìm siêu tham số tốt nhất

### 🚀 Hướng phát triển:

1. **Thử nghiệm thêm mô hình**:
   - Neural Networks
   - Gradient Boosting (XGBoost, LightGBM)
   - Ensemble methods

2. **Kỹ thuật tiền xử lý nâng cao**:
   - Principal Component Analysis (PCA)
   - Feature selection
   - Polynomial features

3. **Tối ưu hóa nâng cao**:
   - RandomizedSearchCV
   - Bayesian Optimization (Optuna)
   - AutoML tools

4. **Triển khai thực tế**:
   - Tạo web app với Streamlit/Flask
   - API với FastAPI
   - Containerize với Docker
   - Deploy lên cloud

5. **Monitoring và MLOps**:
   - Model versioning với MLflow
   - Continuous training
   - A/B testing

---

### 🌟 Lời kết:

Bài toán phân loại hoa Iris tuy đơn giản nhưng là nền tảng tuyệt vời để học Machine Learning. Từ đây, bạn có thể áp dụng kiến thức này vào các bài toán phức tạp hơn trong thực tế!

**Chúc mừng bạn đã hoàn thành dự án Machine Learning đầu tiên! 🎊**